In [ ]:
using TensorKit
import TensorKit.×
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
@show pkgversion(TensorKit)
using JLD2

In [ ]:
# pspace = GradedSpace{fSU₂}((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
pspace = Rep[U₁×SU₂]((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
aspace = Rep[U₁×SU₂]((2, 0) => 2)

# aspacel = aspacer = aspacet = aspaceb = pspace
@show pspace
nothing

In [ ]:
gate = swap_gate(pspace, aspace)
@show space(gate)

In [ ]:
t = id(pspace)
(f1, _) = collect(fusiontrees(t))[1]
f = f1[1]
@show f.uncoupled
@assert f.uncoupled[1] isa U1Irrep

nothing

In [ ]:
a = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
@show space(a)  # [l, t, p; r, b]
@show space(a') # [r, b; l, t, p]

permute!(a, a, ((1,2), (3,4,5)))
@show space(a)
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
pspace = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacel = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacer = aspacel
aspacet = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspaceb = aspacet

Lx = 2
Ly = 2
Ms = Vector{TensorMap}(undef, Lx * Ly)
for ii in 1:Lx*Ly
    Ms[ii] = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
    Ms[ii] = Ms[ii] / norm(Ms[ii])
end
@assert Ms[1] != Ms[2]
ipeps = iPEPS(Ms, Lx, Ly)

envs = iPEPSenv(ipeps)
χ = 10
Nit = 2
nothing

In [ ]:
save("/home/tcmp2/JuliaProjects/testSave.jld2", "ipeps", ipeps, "envs", envs)

In [ ]:
ipeps, envs = load("/home/tcmp2/JuliaProjects/testSave.jld2", "ipeps", "envs")
@show ipeps.Lx
@show envs.Ly
nothing

In [ ]:
using TensorKit
include("./iPEPS_fSU2/iPEPS.jl")
include("./CTMRG_fSU2/CTMRG.jl")
include("./models/Heisenberg_U1.jl")
include("./simple_update_fSU2/simple_update.jl")
include("./Cal_Obs_fSU2/Cal_Obs.jl")

In [ ]:
ipeps, envs, para = load("/home/tcmp2/JuliaProjects/testDiagObs.jld2", "ipeps", "envs", "para")
Obsdiag1 = Cal_Obs_2site(ipeps, envs, ["hijNNN", "SzSz", "SpSm"], para; site1=[1, 1], site2=[2, 2], get_op=get_op_Heisenberg)
Obsdiag1 = Cal_Obs_2site(ipeps, envs, ["hijNNN", "SzSz", "SpSm"], para; site1=[1, 2], site2=[2, 1], get_op=get_op_Heisenberg)